In [39]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
Schema = StructType([StructField('year', IntegerType(), nullable = True),
                        StructField('month', StringType(), True),
                        StructField('day', IntegerType(), True),
                        StructField('weekday', StringType(), True),
                        StructField('hour', IntegerType(), True),
                        StructField('atm_status', StringType(), True),
                        StructField('atm_id', StringType(), True),
                        StructField('atm_manufacturer', StringType(), True),
                        StructField('atm_location', StringType(), True),
                        StructField('atm_streetname', StringType(), True),
                        StructField('atm_street_number', IntegerType(), True),
                        StructField('atm_zipcode', IntegerType(), True),
                        StructField('atm_lat', DoubleType(), True),
                        StructField('atm_lon', DoubleType(), True),
                        StructField('currency', StringType(), True),
                        StructField('card_type', StringType(), True),
                        StructField('transaction_amount', IntegerType(), True),
                        StructField('service', StringType(), True),
                        StructField('message_code', StringType(), True),
                        StructField('message_text', StringType(), True),
                        StructField('weather_lat', DoubleType(), True),
                        StructField('weather_lon', DoubleType(), True),
                        StructField('weather_city_id', IntegerType(), True),
                        StructField('weather_city_name', StringType(), True),
                        StructField('temp', DoubleType(), True),
                        StructField('pressure', IntegerType(), True),
                        StructField('humidity', IntegerType(), True),
                        StructField('wind_speed', IntegerType(), True),
                        StructField('wind_deg', IntegerType(), True),
                        StructField('rain_3h', DoubleType(), True),
                        StructField('clouds_all', IntegerType(), True),
                        StructField('weather_id', IntegerType(), True),
                        StructField('weather_main', StringType(), True),
                        StructField('weather_description', StringType(), True)])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df = spark.read.format("csv")\
                .option("inferSchema", "false")\
                .schema(Schema)\
                .option("header", "false")\
                .load("/user/hadoop/ETL/ATMDATA/part-m-00000")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

In [43]:
df.select('*').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

 The no of row matches 2468572

### Creating the Dimension and Fact tables
##### Creating a dataframe for Location Dimension according to Target Dimension Model

In [44]:
location = df.select("atm_location", "atm_streetname", "atm_street_number", "atm_zipcode", "atm_lat", "atm_lon").distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
location.select('*').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [46]:
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
# creating the primary key column
df_temp = location.rdd.zipWithIndex().toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_temp.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|[Kolding, Vejleve...|  0|
|[Skelagervej 15, ...|  1|
|[Intern HolbÃƒÂ¦k...|  2|
|[Odense, FÃƒÂ¦lle...|  3|
|[Ikast, RÃƒÂ¥dhus...|  4|
|[Randers, ÃƒËœste...|  5|
|[Esbjerg, Strandb...|  6|
|[Lyngby, Jernbane...|  7|
|[Skive, Adelgade,...|  8|
|[Durup, Torvet, 4...|  9|
|[Hadsund, Storega...| 10|
|[NÃƒÂ¦stved, Fari...| 11|
|[Hobro, Adelgade,...| 12|
|[Nordkraft, Kjell...| 13|
|[Slagelse, Marien...| 14|
|[Aars, Himmerland...| 15|
|[Fur, StenÃƒÂ¸re,...| 16|
|[Hasseris, Hasser...| 17|
|[SÃƒÂ¦by, Vesterg...| 18|
|[HelsingÃƒÂ¸r, Sc...| 19|
+--------------------+---+
only showing top 20 rows

In [49]:
dim_location = df_temp.select(col("_1.*"),col("_2").alias('location_id'))
dim_location.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
|    atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|location_id|
+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
|         Kolding|           Vejlevej|              135|       6000| 55.505|  9.457|          0|
|  Skelagervej 15|        Skelagervej|               15|       9000| 57.023|  9.891|          1|
|Intern HolbÃƒÂ¦k|        Slotsvolden|                7|       4300| 55.718| 11.704|          2|
|          Odense|       FÃƒÂ¦lledvej|                3|       5000| 55.394|  10.37|          3|
|           Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|               12|       7430| 56.139|  9.154|          4|
+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
only showing top 5 rows

In [50]:
DIM_LOCATION = dim_location.withColumnRenamed('atm_location','location')\
                            .withColumnRenamed('atm_streetname','streetname')\
                            .withColumnRenamed('atm_street_number','street_number')\
                            .withColumnRenamed('atm_zipcode','zipcode')\
                            .withColumnRenamed('atm_lat','lat')\
                            .withColumnRenamed('atm_lon','lon')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
DIM_LOCATION.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+-------------+-------+------+------+-----------+
|        location|         streetname|street_number|zipcode|   lat|   lon|location_id|
+----------------+-------------------+-------------+-------+------+------+-----------+
|         Kolding|           Vejlevej|          135|   6000|55.505| 9.457|          0|
|  Skelagervej 15|        Skelagervej|           15|   9000|57.023| 9.891|          1|
|Intern HolbÃƒÂ¦k|        Slotsvolden|            7|   4300|55.718|11.704|          2|
|          Odense|       FÃƒÂ¦lledvej|            3|   5000|55.394| 10.37|          3|
|           Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|           12|   7430|56.139| 9.154|          4|
+----------------+-------------------+-------------+-------+------+------+-----------+
only showing top 5 rows

In [52]:
atm = df.select('atm_id', 'atm_manufacturer', 'atm_lat', 'atm_lon')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
atm = atm.withColumnRenamed('atm_id', 'atm_number')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
atm = atm.join(dim_location, on = ['atm_lat', 'atm_lon'], how = "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
atm.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+----------+----------------+------------+--------------------+-----------------+-----------+-----------+
|atm_lat|atm_lon|atm_number|atm_manufacturer|atm_location|      atm_streetname|atm_street_number|atm_zipcode|location_id|
+-------+-------+----------+----------------+------------+--------------------+-----------------+-----------+-----------+
| 57.023|   9.94|        11|             NCR|   Sauersvej|Fridtjof Nansens Vej|                2|       9210|         32|
| 57.023|   9.94|        11|             NCR|   Sauersvej|Fridtjof Nansens Vej|                2|       9210|         32|
| 57.023|   9.94|        11|             NCR|   Sauersvej|Fridtjof Nansens Vej|                2|       9210|         32|
| 57.023|   9.94|        11|             NCR|   Sauersvej|Fridtjof Nansens Vej|                2|       9210|         32|
| 57.023|   9.94|        11|             NCR|   Sauersvej|Fridtjof Nansens Vej|                2|       9210|         32|
| 57.023|   9.94|       

In [56]:
atm = atm.select('atm_number', 'atm_manufacturer', 'location_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
atm = atm.select(col("atm_number").alias("atm_id"),'atm_number', 'atm_manufacturer', 'location_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
atm.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+-----------+
|atm_id|atm_number|atm_manufacturer|location_id|
+------+----------+----------------+-----------+
|    36|        36|             NCR|         61|
|    53|        53|             NCR|         28|
|    30|        30|             NCR|         57|
|    66|        66|             NCR|         20|
|    40|        40| Diebold Nixdorf|         86|
|     7|         7| Diebold Nixdorf|         75|
|    81|        81|             NCR|         81|
|    76|        76|             NCR|         24|
|    49|        49|             NCR|        103|
|    28|        28|             NCR|         33|
|    42|        42|             NCR|         96|
|    45|        45|             NCR|         68|
|   100|       100|             NCR|         72|
|    59|        59| Diebold Nixdorf|         57|
|    95|        95|             NCR|         39|
|    80|        80|             NCR|         30|
|   104|       104|             NCR|         69|
|    79|        79| 

In [68]:
atm.select("*").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

156